In [1]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(f"tensorflow version: {tf.__version__}")


tensorflow version: 2.12.0


In [2]:
train_dr = 'E:/Github Repo/Competition/data/pneumonia_data/chest_xray/train'
val_dr = 'E:/Github Repo/Competition/data/pneumonia_data/chest_xray/val'
test_dr = 'E:/Github Repo/Competition/data/pneumonia_data/chest_xray/test'

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)


In [4]:
val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [5]:
train_data = train_datagen.flow_from_directory(
    train_dr,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

val_data = val_datagen.flow_from_directory(
    val_dr,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_data = test_datagen.flow_from_directory(
    test_dr,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


## Model Development

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',  # Track validation loss for adjustments
    factor=0.5,          # Reduce the learning rate by a factor of 0.5
    patience=3,          # Number of epochs with no improvement before reducing
    min_lr=1e-6          # Minimum learning rate
)


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [8]:
import tensorflow as tf 
import os

# Define a log directory for TensorBoard
log_dir = "logs/fit/" + tf.timestamp().numpy().astype(str)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [9]:
history = model.fit(
    train_data,
    epochs=20,
    validation_data=val_data,
    callbacks=[tensorboard_callback]  # Add TensorBoard callback here
)


Epoch 1/20
163/163 [==============================] - 129s 787ms/step - loss: 0.3029 - accuracy: 0.8823 - val_loss: 0.4428 - val_accuracy: 0.8750
Epoch 2/20
163/163 [==============================] - 118s 721ms/step - loss: 0.2200 - accuracy: 0.9120 - val_loss: 0.8423 - val_accuracy: 0.7500
Epoch 3/20
163/163 [==============================] - 113s 690ms/step - loss: 0.2034 - accuracy: 0.9216 - val_loss: 0.6351 - val_accuracy: 0.7500
Epoch 4/20
163/163 [==============================] - 112s 683ms/step - loss: 0.1881 - accuracy: 0.9256 - val_loss: 1.0461 - val_accuracy: 0.6250
Epoch 5/20
163/163 [==============================] - 108s 660ms/step - loss: 0.1668 - accuracy: 0.9371 - val_loss: 0.3902 - val_accuracy: 0.9375
Epoch 6/20
163/163 [==============================] - 113s 688ms/step - loss: 0.1599 - accuracy: 0.9365 - val_loss: 0.5425 - val_accuracy: 0.8125
Epoch 7/20
163/163 [==============================] - 109s 670ms/step - loss: 0.1520 - accuracy: 0.9429 - val_loss: 0.7541 -

Remove-Item -Recurse -Force .\logs\fit\
py -m tensorboard.main --logdir=logs/fit